# 0. Imports

In [ ]:
%cd ../../..

In [3]:
from sampling.conditional_probability_path import GaussianConditionalProbabilityPath
from sampling.sampleable import PixelArtSampler
from sampling.noise_scheduling import LinearAlpha, LinearBeta
from models.unet import PixelArtUNet
from training.trainer import UnguidedTrainer
from training.evaluation import FID
from training.ema import EMA
from diff_eq.ode_sde import UnguidedVectorFieldODE
from diff_eq.simulator import EulerSimulator
from utils.visualization import visualize_training_logs

import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Train

In [ ]:
if 'unet' in locals():
    del unet
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Initialize probability path
sampler = PixelArtSampler('dataset/images').to(device)
path = GaussianConditionalProbabilityPath(
    p_data=sampler,
    p_simple_shape=[4, 128, 128],
    alpha=LinearAlpha(),
    beta=LinearBeta()
).to(device)

# Initialize model
unet = PixelArtUNet(
    channels=[128, 256, 512, 512],
    num_residual_layers=2,
    t_embed_dim=128,
    midcoder_dropout_p=0.2
)

# Initialize evaluation metric
metric = FID(
    feature=2048,
    normalize=True,
    image_size=(299, 299)
)

# Initialize EMA
ema = EMA(model=unet, max_decay=0.999)

# Initialize trainer
trainer = UnguidedTrainer(
    path=path,
    model=unet,
    experiment_dir="training/experiments/unet_mid_ema",
    eval_metric=metric
)

# Train :D
trainer.train(
    device=device,
    num_epochs=20000,
    batch_size=64,
    lr=1e-4,
    weight_decay=5e-4,
    validate_every=250,
    val_timesteps=200,
    resume=False,
    lr_warmup_steps_frac=0.1,
    num_images_to_save=3,
    save_images_every=250
)

# 2. Visualize model performance over time

In [ ]:
visualize_training_logs(log_path="training/experiments/unet_mid_ema/training_log.csv", scale="linear")

# 3. Evaluate on the test set

In [ ]:
sampler = PixelArtSampler('dataset/images').to(device)

# Initialize model
unet = PixelArtUNet(
    channels=[128, 256, 512, 512],
    num_residual_layers=2,
    t_embed_dim=128,
    midcoder_dropout_p=0.2
).to(device)
unet.eval()

path = GaussianConditionalProbabilityPath(
    p_data=sampler,
    p_simple_shape = [4, 128, 128],
    alpha = LinearAlpha(),
    beta = LinearBeta()
).to(device)
path.eval()

# Initialize evaluation metric
metric = FID(
    feature=2048,
    normalize=True,
    image_size=(299, 299)
)

# Initialize trainer
trainer = UnguidedTrainer(
    path=path,
    model=unet,
    experiment_dir="training/experiments/unet_mid_ema",
    eval_metric=metric
)

test_fid = trainer.evaluate(batch_size=64, device=device, num_timesteps=100, mode="test")

print(f"FID on the test set: {test_fid:.4f}")

# 4. Visualize results

In [ ]:
from utils.helpers import tensor_to_rgba_image

# Parameters
num_samples = 20
num_timesteps = 200

# Load model from checkpoints
cp = torch.load("training/experiments/unet_mid_ema/best_model.pt", map_location=device)
model = PixelArtUNet(
    channels = [128, 256, 512, 512],
    num_residual_layers = 2,
    t_embed_dim = 128,
    midcoder_dropout_p=0.2
).to(device)
model.load_state_dict(cp['model_state'])
model.eval()

sampler = PixelArtSampler('dataset/images').to(device)
path = GaussianConditionalProbabilityPath(
    p_data=sampler,
    p_simple_shape = [4, 128, 128],
    alpha = LinearAlpha(),
    beta = LinearBeta()
).to(device)

# Simulate
ts = torch.linspace(0, 1, num_timesteps).view(1, -1, 1, 1, 1).expand(num_samples, -1, 1, 1, 1).to(device)
x0 = path.p_simple.sample(num_samples).to(device)  # (num_samples, 4, 128, 128)

# Run simulation
ode = UnguidedVectorFieldODE(model)
simulator = EulerSimulator(ode)
x1 = simulator.simulate(x0, ts)  # (num_samples, 4, 128, 128)

for i in range(num_samples):
    img_tensor = x1[i]
    img = tensor_to_rgba_image(img_tensor)  # Expects a tensor in [-1, 1] or [0, 1], shape (4, H, W)
    img.save(f"image_{i}.png")

# Keep RGBA
rgba = x1[:, :4]  # (num_samples, 4, H, W)

# Separate channels
rgb = rgba[:, :3]
alpha = rgba[:, 3:4]  # shape (N, 1, H, W)
rgb = (rgb + 1) / 2  # → [0, 1]
alpha = (alpha + 1) / 2  # → [0, 1]

# Composite over white background
white_bg = torch.ones_like(rgb)  # all ones = white
img = alpha * rgb + (1 - alpha) * white_bg

# Make grid
grid = make_grid(img, nrow=num_samples, normalize=False)

# Plot
plt.figure(figsize=(20, 10))
plt.imshow(grid.permute(1, 2, 0).clamp(0, 1).cpu().numpy())
plt.axis("off")
plt.tight_layout()
plt.show()